In [2]:
import pandas as pd
import geopandas as gpd
import os


In [3]:
folder = r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data"
file_name = r"oregon_rentals.csv"
geocoded_file_name = f"geocoded_{file_name}"

dots_source = pd.read_csv(f"{folder}\{file_name}")

gdf = gpd.GeoDataFrame(
    dots_source, geometry=gpd.points_from_xy(dots_source.Longitude, dots_source.Latitude), crs="EPSG:4326"
)

In [4]:
# demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source\tl_2023_53_bg.shp")

demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")
demographic_areas.to_crs("EPSG:4326", inplace=True)

demographic_areas["GEOID"] = \
    demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)  \
    + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3) \
    + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6) \
    + demographic_areas["BLOCKGROUP_FIPS"].astype(str)

demographic_areas = demographic_areas[["GEOID", "geometry"]].rename(columns={"GEOID":"cbg_geoid"})

cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
cbsa_source.to_crs("EPSG:4326", inplace=True)
cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID":"cbsa_geoid", "NAME": "cbsa_name"})

state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
state_source.to_crs("EPSG:4326", inplace=True)
state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID":"state_id", "State_Name": "state_name"})


In [5]:
geocoded_dots = gdf.sjoin(demographic_areas, how = "left").drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(cbsa_source, how = 'left').drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(state_source, how = 'left').drop(["index_right"], axis=1)

geocoded_dots = geocoded_dots.drop(['geometry', 'Unnamed: 0'],axis=1)
geocoded_dots.to_csv(f"{folder}/{geocoded_file_name}", index= False) 

In [6]:
display(geocoded_dots)

,Property ID,URL,Property Type,Address,City,State,ZIP Code,Country Code,Latitude,Longitude,...,Max Beds,Max Baths,Max Square Feet,Max Rent Price,Description,cbg_geoid,cbsa_geoid,cbsa_name,state_id,State_Code
0,191492758,/ID/Pocatello/1829-W-Quinn-Rd-83202/unit-C/apa...,5,1829 W Quinn Rd Unit C,Pocatello,ID,83202,1,42.904674,-112.491370,...,3,1.0,1080.0,1050.0,NaN,160050015003,38540,"Pocatello, ID",13,ID
1,191492149,/ID/Pocatello/1014-W-Quinn-Rd-83202/unit-3-105...,5,1014 W Quinn Rd Unit 3 105,Pocatello,ID,83202,1,42.905730,-112.471866,...,3,1.0,1080.0,1075.0,NaN,160050015004,38540,"Pocatello, ID",13,ID
2,191491824,/ID/Pocatello/1014-W-Quinn-Rd-83202/unit-3-205...,5,1014 W Quinn Rd Unit 3 205,Pocatello,ID,83202,1,42.905730,-112.471866,...,3,2.0,1080.0,1150.0,NaN,160050015004,38540,"Pocatello, ID",13,ID
3,191491411,/ID/Pocatello/1230-S-2nd-Ave-83201/unit-1/apar...,5,1230 S 2nd Ave Unit 1,Pocatello,ID,83201,1,42.856270,-112.436286,...,2,1.0,NaN,799.0,NaN,160050008001,38540,"Pocatello, ID",13,ID
4,186236886,/ID/Pocatello/1335-S-5th-Ave-83201/unit-104/ap...,5,1335 S 5th Ave Unit 104,Pocatello,ID,83201,1,42.856734,-112.432799,...,2,1.0,NaN,925.0,Discover your new home! Enjoy your newly remod...,160050008001,38540,"Pocatello, ID",13,ID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1172,118575591,/ID/Smelterville/207-Northview-Ave-83868/home/...,6,207 Northview Ave,Smelterville,ID,83868,1,47.542451,-116.177457,...,2,1.0,1168.0,1600.0,Light and airy 1935 home in the heart of the S...,160799603004,NaN,NaN,13,ID
1173,190968330,/ID/Spirit-Lake/4987-Spirit-Lake-Cutoff-Rd-838...,6,4987 Spirit Lake Cutoff Rd,Spirit Lake,ID,83869,1,48.046739,-116.848941,...,3,1.0,1180.0,2000.0,**Charming Country Rental Home in Spirit Lake*...,160179508021,41760,"Sandpoint, ID",13,ID
1174,185660827,/ID/Troy/517-S-Pine-St-83871/unit-1/home/18566...,13,517 S Pine St Unit 1,Troy,ID,83871,1,46.734832,-116.766006,...,4,2.0,NaN,2600.0,"517 S Pine St. Troy, ID 83871 This house has s...",160570056003,34140,"Moscow, ID",13,ID
1175,118606961,/ID/Wallace/59976-Silver-Valley-Rd-83873/home/...,6,59976 Silver Valley Rd,Wallace,ID,83873,1,47.487040,-115.948400,...,2,1.0,NaN,1400.0,"2 bedroom, 1 bath house located in Wallace, Id...",160799604003,NaN,NaN,13,ID
